In [1]:
from Bio import SeqIO
import numpy as np
import pandas as pd

In [2]:
ref_fl = "../data/reference.fasta"

In [3]:
def get_homopolymers(seq_fl):
    """Annotate homoplymer regions of a genome
    Input: fasta file
    Output: Pandas dataframe with
    homopolymer name, star, end, nucleotide, and length
    """
    hmpmer_len = 1
    prev_nuc = ""
    hmpmer = ""
    nuc_pos = 1
    hmpmer_pos = []
    hmpmer_dic = {}
    hu1 = SeqIO.read(seq_fl, "fasta")
    for pos_index, nuc in enumerate(hu1):
        if nuc == prev_nuc:
            hmpmer_len = hmpmer_len + 1
            hmpmer_pos.append(pos_index)
            hmp_name = "hmp" + str(min(hmpmer_pos))
            hmp_nuc = nuc
            hmp_len = max(hmpmer_pos) - min(hmpmer_pos) + 2
            hmpmer_dic[hmp_name] = (
                min(hmpmer_pos),
                max(hmpmer_pos) + 1,
                nuc,
                hmp_len,
                list(
                    np.arange(min(hmpmer_pos) - hmp_len, max(hmpmer_pos) + 2 + hmp_len)
                ),
            )
        else:
            hmpmer_pos = []
            hmpmer_len = 0
        prev_nuc = nuc
    hmp_df = pd.DataFrame.from_dict(
        hmpmer_dic,
        orient="index",
        columns=["start", "end", "nucleotide", "length", "hmp_pos"],
    )
    hmp_df = hmp_df.explode("hmp_pos", ignore_index=True)
    return hmp_df

In [4]:
hmp_tbl = get_homopolymers(ref_fl)
hmp_tbl = hmp_tbl[hmp_tbl['length'] > 3]
hmp_tbl = hmp_tbl.drop_duplicates(['hmp_pos'])
hmp_tbl.head()

,start,end,nucleotide,length,hmp_pos
141,79,82,A,4,75
142,79,82,A,4,76
143,79,82,A,4,77
144,79,82,A,4,78
145,79,82,A,4,79
